In [1]:
import watertap as wt

In [2]:
from case_study_trains import *
import case_study_trains

In [3]:
m = wt.watertap_setup(dynamic = False)

In [4]:
wt.case_study_trains.train = {"case_study": "SantaBarbara",
                             "reference": "NAWI",
                             "scenario": "Baseline"}

In [5]:
# TODO LATER: how to make this sync with info in train input data. We might not need to do that.
#But, if the source water type is different to what is in the train (pfd dictionary), 
#then we should updat the node name. If more than two sources - what to do? Needs to be
#based on pfd node!?

wt.case_study_trains.source_water = {"case_study": "SantaBarbara", 
                             "reference": "NAWI",
                             "scenario": "Baseline",
                             "water_type": "Seawater"}

In [6]:
m = wt.case_study_trains.get_case_study(m = m) # flow is set as case study flow unless defined.

In [7]:
wt.display.show_train2(model_name=m)

In [9]:
wt.run_water_tap(m = m, solver_results = True, print_model_results = True)

degrees_of_freedom: 111
Ipopt 3.12.13: 

******************************************************************************
This program contains Ipopt, a library for large-scale nonlinear optimization.
 Ipopt is released as open source code under the Eclipse Public License (EPL).
         For more information visit http://projects.coin-or.org/Ipopt
******************************************************************************

This is Ipopt version 3.12.13, running with linear solver mumps.
NOTE: Other linear solvers might be more efficient (see Ipopt documentation).

Number of nonzeros in equality constraint Jacobian...:     1868
Number of nonzeros in inequality constraint Jacobian.:        0
Number of nonzeros in Lagrangian Hessian.............:      195

Total number of variables............................:      787
                     variables with only lower bounds:        0
                variables with lower and upper bounds:        3
                     variables with only up

AttributeError: '_ScalarUnitProcess' object has no attribute 'inlet1'

In [10]:
wt.get_results_table(m = m)

,Unit Process Name,Variable,Value,Metric,Unit,Case Study,Scenario
0,sw_onshore_intake,Fixed Capital Investment (FCI),7.903+0.000j,Cost,$MM,SantaBarbara,Baseline
1,sw_onshore_intake,Land,0.012+0.000j,Cost,$MM,SantaBarbara,Baseline
2,sw_onshore_intake,Working Capital,0.395+0.000j,Cost,$MM,SantaBarbara,Baseline
3,sw_onshore_intake,Total Capital Investment (TCI),8.310+0.000j,Cost,$MM,SantaBarbara,Baseline
4,sw_onshore_intake,Catalysts and Chemicals,0.000+0.000j,Annual Cost,$MM/yr,SantaBarbara,Baseline
...,...,...,...,...,...,...,...
983,System,System Electricity,40.334+0.000j,Annual Cost,$MM/yr,SantaBarbara,Baseline
984,System,System Other Variable Operating,0.000+0.000j,Annual Cost,$MM/yr,SantaBarbara,Baseline
985,System,System Fixed Operating,N00000a00000N,Annual Cost,$MM/yr,SantaBarbara,Baseline
986,System,System Total Operating Cost,N00000a00000N,Annual Cost,$MM/yr,SantaBarbara,Baseline


In [17]:
csv_dir = './results/CarlsbadCompare.csv'
json_dir = './tmp/example.json'

wt.app3.run_dash(csv_dir, json_dir)

Dash app running on http://127.0.0.1:8050/


In [18]:
case_study_trains.train["scenario"]

'Baseline'

In [ ]:
cost_range_list = []; #results will be inputted in this array
#up_name = "tri_media_filtration" # which unit process it applies to. TODO hould be user input.

#for value_change in pct_to_target1: # cycles through each value from MC range
for value_change in [0.4, 0.8]: #, 0.9]:

    # create and build model
    m = wt.watertap_setup(dynamic = False)
    m = wt.case_study_trains.get_case_study(name = 'carlsbad', flow = 4.5833, m = m)

    m.fs.tri_media_filtration.water_recovery.fix(value_change)

    # set variable to MC value
    wt.run_water_tap(m)
    results_table = get_results_table(m, unit_process_name)
    cost_range_list.append(results_table.total_up_cost.sum())


In [ ]:
cost_range_list

In [ ]:
#### DO NOT USE THE BELOW ####

In [ ]:
import time
from multiprocessing import Pool
import multiprocessing

mu = 0.6
sigma = .1
num_reps = 50

input_list = np.random.normal(mu,sigma, size = num_reps) #, sigma, num_reps).round(4)

count, bins, ignored = plt.hist(input_list, 25, density=True)
plt.plot(bins, 1/(sigma * np.sqrt(2 * np.pi)) * np.exp( - (bins - mu)**2 / (2 * sigma**2) ),
          linewidth=2, color='r')
plt.show()

### INPUT TO MODEL LIST: ### CAN BE AUTOMATED FOR USER TO LABEL THE VARIABLE. TOOD ###
no_of_proc = 4
list_final = []
for i in range(no_of_proc):
    part2 = len(input_list) / no_of_proc
    i2 = ((i+1)*part2)
    list1 = input_list[int(i*part2):int(i2)]
    list_final.append(list1)
    
    
def monte_run(list_final):
    print('goes in')

    up_name = "tri_media_filtration" # which unit process it applies to. TODO hould be user input.
    cost_range_list = []; #results will be inputted in this array

    #for value_change in pct_to_target1: # cycles through each value from MC range
    for value_change in list_final:

        # create and build model
        m = wt.watertap_setup(dynamic = False)
        m = wt.case_study_trains.get_case_study(name = 'carlsbad', flow = 4.5833, m = m)

        getattr(m.fs, up_name).water_recovery.fix(value_change)

        # set variable to MC value
        result = wt.run_water_tap(m)
        results_table = get_results_table(m, unit_process_names)
        cost_range_list.append(results_table.total_up_cost.sum())


    return cost_range_list

startTime = time.time()

pool=Pool()
dfs = pool.map(monte_run, list_final) #SomeClass().preprocess_data()

executionTime = (time.time() - startTime)
print('Execution time in seconds: ' + str(executionTime))

In [ ]:
####TO DO LOAD AND SAVE!!

In [ ]:
#### SAVE TRAIN ####
# path = 'trains/Tutorial1_treatment_train_example.csv'
# wt.save_train(T, path)

In [ ]:
# #### LOAD TRAIN ####
# path = 'trains/Tutorial1_treatment_train_example.csv'
# TT = wt.load_train(path)

In [ ]:
# wt.display.show_train(TT)